In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

In [3]:
# Read the dataset
data = pd.read_csv('output_theia.csv')

# Encode categorical columns using LabelEncoder
categorical_columns = ['Source UUID', 'Source type', 'Destination UUID', 'Destination type', 'Edge type']
encoder = LabelEncoder()

for col in categorical_columns:
    data[col] = encoder.fit_transform(data[col])

In [4]:
from imblearn.over_sampling import RandomOverSampler

In [5]:
# Split the dataset into features (X) and target (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

oversampler = RandomOverSampler(sampling_strategy='minority')
X, y = oversampler.fit_resample(X, y)

# Normalize the input features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode the target variable (Label)
y = LabelEncoder().fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


In [6]:
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
# Calculate class weights
sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)


In [7]:
# Create a neural network model
def create_model():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Compile the model with a learning rate of 0.001
optimizer = Adam(learning_rate=0.0001)


In [ ]:
# Number of clients
num_clients = 5
client_models = [create_model() for _ in range(num_clients)]

for model in client_models:
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=1e-6)
history = model.fit(X_train, y_train, epochs=30, batch_size=64, verbose=1, validation_split=0.1, callbacks=[reduce_lr], sample_weight=sample_weights)
# Number of rounds
num_rounds = 3
epochs_per_round = 3

In [ ]:
for r in range(num_rounds):
    print(f"Round {r + 1}/{num_rounds}")
    for i, model in enumerate(client_models):
        print(f"Training client {i + 1}/{num_clients}")
        model.fit(X_train, y_train, epochs=epochs_per_round, batch_size=64, verbose=1, validation_split=0.1, callbacks=[reduce_lr])

    # Average the clients' model weights
    avg_weights = [client_models[0].get_weights()]
    for i in range(1, num_clients):
        weights = client_models[i].get_weights()
        for j in range(len(weights)):
            avg_weights[0][j] += weights[j]
    for j in range(len(avg_weights[0])):
        avg_weights[0][j] /= num_clients

    # Set the new average weights to all client models
    for model in client_models:
        model.set_weights(avg_weights[0])


In [ ]:
import numpy as np

# Evaluate the global model
global_model = create_model()
global_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
global_model.set_weights(avg_weights[0])
_, accuracy = global_model.evaluate(X_test, y_test)
print(f"Global model accuracy: {accuracy * 100:.2f}%")

# Get predictions
y_pred_probs = global_model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int).reshape(-1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")